<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/riva_tts_tts-python-advanced-customization-with-ssml/nvidia_logo.png" style="width: 90px; float: right;">

# How do I customize Riva TTS audio output with SSML?

This tutorial walks you through some of the advanced features for customization of Riva TTS audio output with Speech Synthesis Markup Language (SSML).

## NVIDIA Riva Overview

NVIDIA Riva is a GPU-accelerated SDK for building Speech AI applications that are customized for your use case and deliver real-time performance. <br/>
Riva offers a rich set of speech and natural language understanding services such as:

- Automated speech recognition (ASR)
- Text-to-Speech synthesis (TTS)
- A collection of natural language processing (NLP) services, such as named entity recognition (NER), punctuation, and intent classification.

In this tutorial, we will customize Riva TTS audio output with SSML. <br> 
To understand the basics of Riva TTS APIs, refer to [How do I use Riva TTS APIs with out-of-the-box models?](https://github.com/nvidia-riva/tutorials/tree/stable/tts-python-basics.ipynb). <br>

For more information about Riva, refer to the [Riva developer documentation](https://developer.nvidia.com/riva).

## Customizing Riva TTS audio output with SSML

Speech Synthesis Markup Language (SSML) specification is a markup for directing the performance of the virtual speaker. Riva supports portions of SSML, allowing you to adjust pitch, rate, and pronunciation of the generated audio.  
SSML support is available only for the FastPitch model at this time. The FastPitch model must be exported using NeMo>=1.5.1 and the nemo2riva>=1.8.0 tool.

All SSML inputs must be a valid XML document and use the <speak> root tag. All non-valid XML and all valid XML with a different root tag are treated as raw input text.

Riva TTS supports the following SSML tags:  

- The ``prosody`` tag, which supports attributes ``rate``, ``pitch`` and ``volume``, through which we can control the rate, pitch and volume of the generated audio.  

- The ``phoneme`` tag, which allows us to control the pronunciation of the generated audio.
    
- The ``sub`` tag, which allows us to replace the pronounciation of the specified word or phrase with a different word or phrase.
    
Let's look at customization of Riva TTS with these SSML tags in some detail.

#### Requirements and setup

1. Start the Riva Speech Skills server.  
Follow the instructions in the [Riva Quick Start Guide](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html#) to deploy OOTB TTS models on the Riva Speech Skills server before running this tutorial.  

2. Install the additional Python libraries to run this tutorial.  
Run the following commands to install the libraries:

In [ ]:
# We need numpy to read the output from Riva TTS request
!pip install numpy
!pip install nvidia-riva-client

#### Import Riva Client Libraries

Let's first import some required libraries, including the Riva Client libraries:

In [ ]:
import numpy as np
import IPython.display as ipd
import grpc

import riva.client

#### Create Riva Clients and connect to the Riva Speech API server

The below URI assumes a local deployment of the Riva Speech API server is on the default port. In case the server deployment is on a different host or via Helm chart on Kubernetes, use an appropriate URI.

In [ ]:
auth = riva.client.Auth(uri='localhost:50051')

riva_tts = riva.client.SpeechSynthesisService(auth)

In [ ]:
sample_rate_hz = 44100
req = { 
        "language_code"  : "en-US",
        "encoding"       : riva.client.AudioEncoding.LINEAR_PCM ,
        "sample_rate_hz" : sample_rate_hz,
        "voice_name"     : "English-US.Female-1" 
}

### Customizing rate, pitch and volume with the `prosody` tag

#### Pitch Attribute
Riva supports an additive relative change to the pitch. The `pitch` attribute has a range of [-3, 3]. Values outside this range result in an error being logged and no audio returned. This value returns a pitch shift of the attribute value multiplied with the speaker’s pitch standard deviation when the FastPitch model is trained. For the pretrained checkpoint that was trained on LJSpeech, the standard deviation was 52.185. For example, a pitch shift of 1.25 results in a change of 1.25*52.185=~65.23Hz pitch shift up. 
Riva also supports the prosody tags as per the SSML specs. Prosody tags `x-low`, `low`, `medium`, `high`, `x-high`, and `default` are supported.

The `pitch` attribute is expressed in the following formats:
- `pitch="1"`
- `pitch="+1.8"`
- `pitch="-0.65"`
- `pitch="high"`
- `pitch="default"`

For the pretrained Female-1 checkpoint, the standard deviation is 53.33 Hz.
For the pretrained Male-1 checkpoint, the standard deviation is 47.15 Hz.

The `pitch` attribute does not support `Hz`, `st`, and `%` changes. Support is planned for a future Riva release.

#### Rate Attribute
Riva supports a percentage relative change to the rate. The `rate` attribute has a range of [25%, 250%]. Values outside this range result in an error being logged and no audio returned. 
Riva also supports the prosody tags as per the SSML specs. Prosody tags `x-low`, `low`, `medium`, `high`, `x-high`, and `default` are supported.

The `rate` attribute is expressed in the following formats:
- `rate="35%"`
- `rate="+200%"`
- `rate="low"`
- `rate="default"`

#### Volume Attribute

Riva supports the volume attribute as described in the SSML specs. The volume attribute supports a range of [-13, 8]dB. Values outside this range result in an error being logged and no audio returned. Tags silent, x-soft, soft, medium, loud, x-loud, and default are supported.

The volume attribute is expressed in the following formats:

- `volume="+1dB"`
- `volume="-5.7dB"`
- `volume="x-loud"`
- `volume="default"`


Let's look at an example showing these pitch, rate and volume customizations for Riva TTS:

In [ ]:
"""
    Raw text is "Today is a sunny day. But it might rain tomorrow."
    We are updating this raw text with SSML:
    1. Envelope raw text in '<speak>' tags as is required for SSML
    2. Add '<prosody>' tag with 'pitch' attribute set to '2.5'
    3. Add '<prosody>' tag with 'rate' attribute set to 'high'
    4. Add '<volume>' tag with 'volume' attribute set to '+1dB'
"""
raw_text = "Today is a sunny day. But it might rain tomorrow."
ssml_text = """<speak><prosody pitch='2.5'>Today is a sunny day</prosody>. <prosody rate='high' volume='+1dB'>But it might rain tomorrow.</prosody></speak>"""
print("Raw Text: ", raw_text)
print("SSML Text: ", ssml_text)


req["text"] = ssml_text
# Request to Riva TTS to synthesize audio
resp = riva_tts.synthesize(**req)

# Playing the generated audio from Riva TTS request
audio_samples = np.frombuffer(resp.audio, dtype=np.int16)
ipd.display(ipd.Audio(audio_samples, rate=sample_rate_hz))

#### Expected results if you run the notebook:
`<prosody pitch='2.5'>Today is a sunny day</prosody>. <prosody rate='high' volume='+1dB'>But it might rain tomorrow.</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_0.wav" type="audio/ogg"></audio>  
##### Note: If the audio controls are not seen in the notebook. Open the notebook in github dev or view it in the [riva docs](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/tts/tts-ssml.html)


Here are more examples showing the effects of changes in pitch, and rate attribute values on the generated audio:

In [ ]:
# SSML texts we want to try
ssml_texts = [
  """<speak>This is a normal sentence</speak>""",
  """<speak><prosody pitch="0." rate="100%">This is also a normal sentence</prosody></speak>""",
  """<speak><prosody rate="200%">This is a fast sentence</prosody></speak>""",
  """<speak><prosody rate="60%">This is a slow sentence</prosody></speak>""",
  """<speak><prosody pitch="+1.0">Now, I'm speaking a bit higher</prosody></speak>""",
  """<speak><prosody pitch="-0.5">And now, I'm speaking a bit lower</prosody></speak>""",
  """<speak>S S M L supports <prosody pitch="-1">nested tags. So I can speak <prosody rate="150%">faster</prosody>, <prosody rate="75%">or slower</prosody>, as desired.</prosody></speak>""",
  """<speak><prosody volume='x-soft'>I'm speaking softly.</prosody><prosody volume='x-loud'> And now, This is loud.</prosody></speak>""",
]

# Loop through 'ssml_texts' list and synthesize audio with Riva TTS for each entry 'ssml_texts'
for ssml_text in ssml_texts:
    req["text"] = ssml_text
    resp = riva_tts.synthesize(**req)
    audio_samples = np.frombuffer(resp.audio, dtype=np.int16)
    print("SSML Text: ", ssml_text)
    ipd.display(ipd.Audio(audio_samples, rate=sample_rate_hz))
    print("--------------------------------------------")

#### Expected results if you run the notebook:
`This is a normal sentence`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_1.wav" type="audio/ogg"></audio>  

`<prosody pitch="0." rate="100%">This is also a normal sentence</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_2.wav" type="audio/ogg"></audio> 

`<prosody rate="200%">This is a fast sentence</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_3.wav" type="audio/ogg"></audio> 

`<prosody rate="60%">This is a slow sentence</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_4.wav" type="audio/ogg"></audio> 

`<prosody pitch="+1.0">Now, I'm speaking a bit higher</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_5.wav" type="audio/ogg"></audio> 

`<prosody pitch="-0.5">And now, I'm speaking a bit lower</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_6.wav" type="audio/ogg"></audio> 

`S S M L supports <prosody pitch="-1">nested tags. So I can speak <prosody rate="150%">faster</prosody>, <prosody rate="75%">or slower</prosody>, as desired.</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_7.wav" type="audio/ogg"></audio> 

`<prosody volume='x-soft'>I'm speaking softly.</prosody><prosody volume='x-loud'> And now, This is loud.</prosody>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_8.wav" type="audio/ogg"></audio>


### Customizing pronunciation with the `phoneme` tag

We can use the `phoneme` tag to override the pronunciation of words from the predicted pronunciation. For a given word or sequence of words, use the `ph` attribute to provide an explicit pronunciation, and the `alphabet` attribute to provide the phone set.  
Currently, only `x-arpabet` is supported for pronunciation dictionaries based on CMUdict. IPA support will be added soon.

The full list of phonemes in the CMUdict can be found at [cmudict.phone](https://github.com/cmusphinx/cmudict/blob/master/cmudict.phones). The list of supported symbols with stress can be found at [cmudict.symbols](https://github.com/cmusphinx/cmudict/blob/master/cmudict.symbols). For a mapping of these phones to English sounds, refer to the [ARPABET Wikipedia page](https://en.wikipedia.org/wiki/ARPABET).

Let's look at an example showing this custom pronunciation for Riva TTS:

In [ ]:
# Setting up Riva TTS request with SynthesizeSpeechRequest
"""
    Raw text is "You say tomato, I say tomato."
    We are updating this raw text with SSML:
    1. Envelope raw text in '<speak>' tags as is required for SSML
    2. For a substring in the raw text, add '<phoneme>' tags with 'alphabet' attribute set to 'x-arpabet' 
       (currently the only supported value) and 'ph' attribute set to a custom pronunciation based on CMUdict and ARPABET

"""
raw_text = "You say tomato, I say tomato."
ssml_text = '<speak>You say <phoneme alphabet="x-arpabet" ph="{@T}{@AH0}{@M}{@EY1}{@T}{@OW2}">tomato</phoneme>, I say <phoneme alphabet="x-arpabet" ph="{@T}{@AH0}{@M}{@AA1}{@T}{@OW2}">tomato</phoneme>.</speak>'

print("Raw Text: ", raw_text)
print("SSML Text: ", ssml_text)

req["text"] = ssml_text
# Request to Riva TTS to synthesize audio
resp = riva_tts.synthesize(**req)

# Playing the generated audio from Riva TTS request
audio_samples = np.frombuffer(resp.audio, dtype=np.int16)
ipd.display(ipd.Audio(audio_samples, rate=sample_rate_hz))

#### Expected results if you run the notebook:
`<speak>You say <phoneme alphabet="x-arpabet" ph="{@T}{@AH0}{@M}{@EY1}{@T}{@OW2}">tomato</phoneme>, I say <phoneme alphabet="x-arpabet" ph="{@T}{@AH0}{@M}{@AA1}{@T}{@OW2}">tomato</phoneme>.`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_9.wav" type="audio/ogg"></audio> 


Here are more examples showing the customization of pronunciation in generated audio:

In [ ]:
# SSML texts we want to try
ssml_texts = [
  """<speak>You say <phoneme alphabet="x-arpabet" ph="{@D}{@EY1}{@T}{@AH0}">data</phoneme>, I say <phoneme alphabet="x-arpabet" ph="{@D}{@AE1}{@T}{@AH0}">data</phoneme>.</speak>""",
  """<speak>Some people say <phoneme alphabet="x-arpabet" ph="{@R}{@UW1}{@T}">route</phoneme> and some say <phoneme alphabet="x-arpabet" ph="{@R}{@AW1}{@T}">route</phoneme>.</speak>""",
]

# Loop through 'ssml_texts' list and synthesize audio with Riva TTS for each entry 'ssml_texts'
for ssml_text in ssml_texts:
    req["text"] = ssml_text
    resp = riva_tts.synthesize(**req)
    audio_samples = np.frombuffer(resp.audio, dtype=np.int16)
    print("SSML Text: ", ssml_text)
    ipd.display(ipd.Audio(audio_samples, rate=sample_rate_hz))
    print("--------------------------------------------")

#### Expected results if you run the notebook:
`You say <phoneme alphabet="x-arpabet" ph="{@D}{@EY1}{@T}{@AH0}">data</phoneme>, I say <phoneme alphabet="x-arpabet" ph="{@D}{@AE1}{@T}{@AH0}">data</phoneme>`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_10.wav" type="audio/ogg"></audio>

`Some people say <phoneme alphabet="x-arpabet" ph="{@R}{@UW1}{@T}">route</phoneme> and some say <phoneme alphabet="x-arpabet" ph="{@R}{@AW1}{@T}">route</phoneme>.`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_11.wav" type="audio/ogg"></audio>

### Replacing pronunciation with the `sub` tag

We can use the `sub` tag to replace the pronounciation of the specified word or phrase with a different word or phrase. You can specify the pronunciation to substitute with the alias attribute.


In [ ]:
# Setting up Riva TTS request with SynthesizeSpeechRequest
"""
    Raw text is "WWW is know as the web"
    We are updating this raw text with SSML:
    1. Envelope raw text in '<speak>' tags as is required for SSML
    2. Add '<sub>' tag with 'alias' attribute set to replace www with `World Wide Web`

"""
raw_text = "WWW is know as the web."
ssml_text = '<speak><sub alias="World Wide Web">WWW</sub> is known as the web.</speak>'

print("Raw Text: ", raw_text)
print("SSML Text: ", ssml_text)

req["text"] = ssml_text
# Request to Riva TTS to synthesize audio
resp = riva_tts.synthesize(**req)
# Playing the generated audio from Riva TTS request
audio_samples = np.frombuffer(resp.audio, dtype=np.int16)
ipd.display(ipd.Audio(audio_samples, rate=sample_rate_hz))

#### Expected results if you run the notebook:
`<sub alias="World Wide Web">WWW</sub> is known as the web.`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_12.wav" type="audio/ogg"></audio>


### Emphasize words with the `emphasis` tag

Use the emphasis tag to emphasize words. Use riva-build with enable_emphasis_tag, start_of_emphasis_token, and end_of_emphasis_token to enable the emphasis feature. 
The emphasis tag should be used per word basis. If the word ends with a punctuation, only the word will be emphasized and not the punctuation.

#### Limitation
The emphasis tag is training data dependent and is available only in a few models. The models which are trained without the emphasis tag in the training data will not result in emphasized speech. Input text containing more than one word wrapped by the emphasis tag is an invalid input. Space wrapped inside the emphasis tag is also an invalid input.

<div class="alert alert-block alert-warning"> Warning: The emphasis tag feature does not support nesting of other SSML tags inside it. The emphasis tag does not support the level attribute. </div>

In [ ]:
# Setting up Riva TTS request with SynthesizeSpeechRequest
"""
    Raw text is "Hello World"
    We are updating this raw text with SSML:
    1. Envelope raw text in '<speak>' tags as is required for SSML
    2. Add '<emphasis>' tag around `love`

"""
ssml_texts = [
   """<speak>I would <emphasis>love</emphasis> to try that.</speak>""",
   """<speak><emphasis>Wow!</emphasis> Thats really cool.</speak>"""
]

print("Raw Text: ", raw_text)
print("SSML Text: ", ssml_text)

for ssml_text in ssml_texts:
    req["text"] = ssml_text
    resp = riva_tts.synthesize(**req)
    # Playing the generated audio from Riva TTS request
    audio_samples = np.frombuffer(resp.audio, dtype=np.int16)
    print("SSML Text: ", ssml_text)
    ipd.display(ipd.Audio(audio_samples, rate=sample_rate_hz))
    print("--------------------------------------------")

#### Expected results if you run the notebook:
`I would <emphasis>love</emphasis> to try that.`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_13.wav" type="audio/ogg"></audio>

`<emphasis>Wow!</emphasis> Thats really cool.`  
<audio controls src="https://raw.githubusercontent.com/nvidia-riva/tutorials/stable/audio_samples/tts_samples/ssml_sample_14.wav" type="audio/ogg"></audio>

Information about customizing Riva TTS with SSML can also be found in the documentation [here](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/tts/tts-ssml.html#). 

## Go deeper into Riva capabilities

### Additional Riva tutorials

Checkout more Riva TTS (and ASR) tutorials [here](https://github.com/nvidia-riva/tutorials). These tutorials provide a deeper understanding of the advanced features of Riva TTS, including customizing TTS for your specific needs.

### Sample applications

Riva comes with various sample applications. They demonstrate how to use the APIs to build applications such as a [chatbot](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/samples/weather.html), a domain specific speech recognition, [keyword (entity) recognition system](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/samples/callcenter.html), or simply how Riva allows scaling out for handling massive amounts of requests at the same time. Refer to ([SpeechSquad)](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/samples/speechsquad.html) for more information.  
Refer to the *Sample Application* section in the [Riva developer documentation](https://developer.nvidia.com/) for more information.


###  Riva Automated Speech Recognition (ASR)

Riva's ASR offering comes with OOTB pipelines for English, German, Spanish, Russian and Mandarin. It can be used in streaming or batch inference modes and easily deployed using the [Riva Quick Start scripts](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html). Follow [this link](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/asr/asr-overview.html) to better understand Riva's ASR capabilities. Explore how to use Riva ASR APIs with the OOTB voices with [this Riva ASR tutorial](https://github.com/nvidia-riva/tutorials/blob/dev/22.04/asr-python-basics.ipynb).


### Additional resources

For more information about each of the APIs and their functionalities, refer to the [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/protobuf-api/protobuf-api-root.html).